<a href="https://colab.research.google.com/github/Sylar257/My-data-science-tool-kit/blob/master/FastAI_NLP_deeper_dive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [0]:
bs = 64

In [2]:
# this is the sample data which only has a single `csv` file.
path_sample = untar_data(URLs.IMDB_SAMPLE)
path_sample.ls()

[PosixPath('/root/.fastai/data/imdb_sample/texts.csv')]

In [11]:
data_lm_sample = TextDataBunch.from_csv(path_sample, 'texts.csv')
data_lm_sample.show_batch

<bound method DataBunch.show_batch of TextClasDataBunch;

Train: LabelList (799 items)
x: TextList
xxbos xxmaj terry xxmaj gilliam 's stunning feature - length adaptation of xxmaj chris xxmaj marker 's short film xxup la xxup xxunk is full of mind - xxunk surprises , yet still touches your heart thanks to the superb cast . xxmaj gilliam 's flair for the xxunk works with the script by xxmaj david and xxmaj janet xxmaj peoples to play with your head as much as it does with poor xxmaj james xxmaj cole ( xxmaj willis at his most xxmaj steve mcqueen - like -- better than mcqueen , even ! ) , a time - traveling convict from the future who literally does n't know whether he 's coming or going as a team of scientists keeps sending him back to the wrong eras while trying to prevent a 1995 plague that 's deadly to humans but harmless to animals . xxmaj willis , the xxunk xxmaj oscar - nominated xxmaj brad xxmaj pitt , and xxmaj madeline xxmaj xxunk as a well - meaning psychiatrist give some of t

In [4]:
# Full data, we are going to make use of `train`, `test` and `unsup`
path = untar_data(URLs.IMDB)
path.ls()

[PosixPath('/root/.fastai/data/imdb/train'),
 PosixPath('/root/.fastai/data/imdb/unsup'),
 PosixPath('/root/.fastai/data/imdb/tmp_clas'),
 PosixPath('/root/.fastai/data/imdb/test'),
 PosixPath('/root/.fastai/data/imdb/README'),
 PosixPath('/root/.fastai/data/imdb/tmp_lm'),
 PosixPath('/root/.fastai/data/imdb/imdb.vocab')]

In [0]:
data_lm = (TextList.from_folder(path).filter_by_folder(include=['train','test','unsup']).split_by_rand_pct(0.1, seed=42).label_for_lm().databunch(bs=bs))

In [8]:
data_lm.show_batch()

idx,text
0,"remake was done well , and it was kind of a guilty pleasure for me . xxmaj bloody , intense , and great special effects . xxmaj in short , a great popcorn movie for any horror fan . \n \n xxmaj which brings us to "" xxmaj the xxmaj hills have eyes 2 "" . xxmaj man , where do i start ? xxmaj the plot , or"
1,"there is precious little beyond a short . \n \n xxmaj after an hour we saw variations of the same scene over and over again . i nearly screamed at the screen , "" xxmaj we get it , we get it xxrep 5 ! "" xxmaj it 's amazing that after that left the theatre , we could drive home , watch the xxmaj daily xxmaj show and"
2,stuff is doing my head in . \n \n xxmaj richard xxmaj donner had the idea of ' keep it real ' for his 2 outings . xxmaj and i do find his approach to the special and optical effects to be the most appropriate . xxmaj brian xxmaj singer bombards us with so much xxup cgi that it really takes you out of the story and constantly reminds
3,"future with the class of ' 71 "" ) , the xxmaj phoenician city - where they wear xxmaj roman xxmaj imperial armor but which inexplicably has xxmaj egyptian xxunk xxunk xxunk xxmaj xxunk invented the alphabet - come xxunk worst of all , xxmaj macallum and xxmaj ekland ( with her fright wig ) playing smoochy - face -oh the horror ! xxmaj the best parts are that the"
4,", the younger , has a major cocaine addiction . xxmaj adam , the eldest , is a doctor . xxmaj this movie is suppose to show the plunge from reality to the extreme lows that drugs make possible . xxmaj it however , does not . xxmaj it shows that cocaine can be fun no matter what the situation happens to be present . xxmaj most of the movie"


In [9]:
len(data_lm.vocab.itos),len(data_lm.train_ds),len(data_lm.valid_ds)

(60000, 90000, 10000)

In [7]:
len(data_lm_sample.vocab.itos),len(data_lm_sample.train_ds),len(data_lm_sample.valid_ds)

NameError: ignored

In [10]:
data_lm_sample.batch_size

NameError: ignored

In [0]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=1.).to_fp16()

In [12]:
lr = 1e-2
lr *= bs/48; lr

0.013333333333333332

In [0]:
learn_lm.fit_one_cycle(1, lr, moms=[0.8, 0.7])

epoch,train_loss,valid_loss,accuracy,time


In [0]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(10, lr/10, moms=(0.8,0.7))

In [0]:
learn_lm.save_encoder('fine_tuned_enc_10')

In [0]:
data_clas = (TextList.from_folder(path, vocab = data_lm.vocab)
             .split_by_folder(valid='test')
             .label_from_folder(classes=['pos','neg'])
             .databunch(bs=bs))

In [0]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult = 0.5).to_fp16()
learn_c.load_encoder('fine_tuned_enc_10')
learn_c.freeze()

In [0]:
lr = 2e-2
lr *= bs/48

In [0]:
learn_c.fit_one_cycle(1, lr, moms=(0.8,0.7))

In [0]:
learn_c.save('1')

In [0]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

In [0]:
learn_c.save('2')

In [0]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(1, slice(lr/2/(2.6**4),lr/2)

In [0]:
learn_c.save('3')

In [0]:
learn_c.unfreeze()
learn_c.fit_one_cycle(2, slice(lr/10/(2.6**4),lr/10)